In [1]:
# !mkdir data
# !wget -O data/fhv_tripdata_2021-01.parquet https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
# !wget -O data/fhv_tripdata_2021-02.parquet https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet

In [2]:
import pandas as pd

# Q1. Downloading the data
---
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

In [3]:
jan_df = pd.read_parquet("data/fhv_tripdata_2021-01.parquet")
jan_df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037
...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285


**Solution**: 1154112 (item 2)

# Q2. Computing duration
---
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the average trip duration in January?

In [4]:
jan_df["duration"] = (jan_df.dropOff_datetime - jan_df.pickup_datetime).apply(lambda dt: dt.total_seconds() / 60)
jan_df.duration.mean()

19.1672240937939

**Solution:** 19.17 (item 2)

In [5]:
jan_df = jan_df[jan_df.duration > 1]
jan_df = jan_df[jan_df.duration < 60]

In [6]:
jan_df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


In [7]:
1154112 - 1106890

47222

**Solution:** 47222

# Q3. Missing values
---
The features we'll use for our model are the pickup and dropoff location IDs.

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

In [8]:
jan_df.PUlocationID.isna().sum() / jan_df.shape[0] * 100

83.63125513827029

**Solution:** 83.63% (item 4)

# Q4. One-hot encoding
---
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries
- Fit a dictionary vectorizer
- Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns).

In [9]:
jan_df.PUlocationID.fillna(-1, inplace=True)
jan_df.DOlocationID.fillna(-1, inplace=True)
jan_df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,-1.0,-1.0,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,-1.0,-1.0,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,-1.0,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,-1.0,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,-1.0,71.0,None,B00037,9.050000
...,...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266,8.750000
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,,57.600000
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285,16.200000
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285,19.433333


In [10]:
from sklearn.feature_extraction import DictVectorizer

In [11]:
jan_dict = jan_df[["PUlocationID", "DOlocationID"]].astype(str).to_dict(orient="records")

In [12]:
dv = DictVectorizer()
X_train = dv.fit_transform(jan_dict)

In [13]:
X_train

<1106890x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2213780 stored elements in Compressed Sparse Row format>

**Solution:** 525 (item 4)

# Q5. Training a model
---
Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [14]:
from sklearn.linear_model import LinearRegression

In [15]:
y_train = jan_df.duration.values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred = lr.predict(X_train)

In [18]:
from sklearn.metrics import mean_squared_error

In [19]:
mean_squared_error(y_train, y_pred, squared=False)

10.387902642650943

**Solution:** 10.38 (item 2)

# Q6. Evaluating the model
---
Now let's apply this model to the validation dataset (Feb 2021).

What's the RMSE on validation?

In [20]:
feb_df = pd.read_parquet("data/fhv_tripdata_2021-02.parquet")
feb_df["duration"] = (feb_df.dropOff_datetime - feb_df.pickup_datetime).apply(lambda dt: dt.total_seconds() / 60)
feb_df = feb_df[feb_df.duration > 1]
feb_df = feb_df[feb_df.duration < 60]
feb_df.PUlocationID.fillna(-1, inplace=True)
feb_df.DOlocationID.fillna(-1, inplace=True)
feb_df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021,7.950000
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1.0,225.0,None,B00037,13.800000
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1.0,61.0,None,B00037,8.966667
...,...,...,...,...,...,...,...,...
1037687,B03282,2021-02-28 23:01:16,2021-02-28 23:14:48,-1.0,31.0,None,B01717,13.533333
1037688,B03282,2021-02-28 23:36:10,2021-02-28 23:47:38,-1.0,169.0,None,B01717,11.466667
1037689,B03285,2021-02-28 23:18:36,2021-02-28 23:43:59,28.0,171.0,None,B03285,25.383333
1037690,B03285,2021-02-28 23:26:34,2021-02-28 23:44:37,16.0,252.0,None,B03285,18.050000


In [21]:
feb_dict = feb_df[["PUlocationID", "DOlocationID"]].astype(str).to_dict(orient="records")
X_val = dv.transform(feb_dict)

In [22]:
X_val

<987770x525 sparse matrix of type '<class 'numpy.float64'>'
	with 1975537 stored elements in Compressed Sparse Row format>

In [23]:
y_val = feb_df.duration.values

In [24]:
y_preds = lr.predict(X_val)

In [25]:
mean_squared_error(y_val, y_preds, squared=False)

10.897037731359287

**Solution:** 11.20 (item 2)